In [1]:
import random
import string
import numpy as np

### Config

In [2]:
DISH_COUNT = 300
MIN_CALORIES_PER_DISH = 100
MAX_CALORIES_PER_DISH = 1200

DAYS_PER_MENU = 7
DISHES_PER_DAY = 3

POPULATION_SIZE = 20
MATING_POPULATION_SIZE = 10
ITERATIONS = 1000

CALORIES_PER_DAY = 2000

#### Test data

In [3]:
dishes = {}
for _ in range(DISH_COUNT):
    name = ''.join(random.choices(string.ascii_lowercase, k=6))
    dishes[name] = random.randint(MIN_CALORIES_PER_DISH, MAX_CALORIES_PER_DISH)
dishes = list(dishes.items())
    
print(dishes[:7])

[('yjknwx', 186), ('ssbskk', 774), ('twxlia', 691), ('agnmek', 545), ('nucsgd', 809), ('rmkevn', 101), ('easeba', 473)]


#### Sample random menu

In [4]:
def create_random_menu():
    menu = np.ndarray((7,3), tuple)
    for day in range(DAYS_PER_MENU):
        for dish_index in range(DISHES_PER_DAY):
            menu[day, dish_index] = random.choice(dishes)
    return menu

In [5]:
def count_calories_per_day(menu):
    calories = []
    for day in range(DAYS_PER_MENU):
        calories.append(0)
        for dish_index in range(DISHES_PER_DAY):
            calories[-1] += menu[day, dish_index][1]
    return calories

def score_menu(menu):
    score = 0
    differences = [calories - CALORIES_PER_DAY for calories in count_calories_per_day(menu)]
    squares = [calories**2 for calories in differences]
    return sum(squares)

In [6]:
sample_menu = create_random_menu()
print(sample_menu)
print()
print(count_calories_per_day(sample_menu))
print("score: " + str(score_menu(sample_menu)))

[[('btpith', 223) ('sbvbbr', 1190) ('woibew', 223)]
 [('zajfsy', 209) ('kqyqvq', 1192) ('sdxjgo', 309)]
 [('tjurzk', 439) ('pmlbhd', 576) ('pdskxh', 564)]
 [('ptkweb', 835) ('jeztav', 724) ('oqfszs', 176)]
 [('mzfyid', 1020) ('xknjxq', 139) ('envogq', 1085)]
 [('hoqprh', 473) ('gkiqil', 928) ('tjurzk', 439)]
 [('bqflvi', 138) ('rmukfn', 1166) ('cdjlrz', 1001)]]

[1636, 1710, 1579, 1735, 2244, 1840, 2305]
score: 642223


In [7]:
def crossover(first_menu, second_menu):
    child = first_menu.copy()
    for day_index in range(DAYS_PER_MENU):
        if random.choice((0, 1)) == 1:
            child[day_index] = second_menu[day_index]
    return child

def random_mutate(menu):
    menu[random.randint(0, DAYS_PER_MENU-1), random.randint(0, DISHES_PER_DAY-1)] = random.choice(dishes)
    return menu
    
def swap_mutate(menu):
    first_day_index = random.randint(0, DAYS_PER_MENU-1)
    second_day_index = random.randint(0, DAYS_PER_MENU-1)
    dish_index = random.randint(0, DISHES_PER_DAY-1)
    menu[first_day_index, dish_index], menu[second_day_index, dish_index] = menu[second_day_index, dish_index], menu[first_day_index, dish_index]
    return menu

In [8]:
population = []
for _ in range(POPULATION_SIZE):
    population.append(create_random_menu())

In [9]:
for i in range(ITERATIONS):
    population.sort(key=score_menu)
    mating_population = population[:MATING_POPULATION_SIZE]

    offspring_population = []
    for _ in range(POPULATION_SIZE - MATING_POPULATION_SIZE):
        offspring_population.append(crossover(random.choice(mating_population), random.choice(mating_population)))
        if random.random() < 0.1:
            offspring_population[-1] = random_mutate(offspring_population[-1])
        if random.random() < 0.1:
            offspring_population[-1] = swap_mutate(offspring_population[-1])

    population = mating_population + offspring_population

In [10]:
population.sort(key=score_menu)
print(population[0])
print(count_calories_per_day(population[0]))
print(score_menu(population[0]))

[[('hcwqie', 124) ('vhgwfc', 864) ('ybswnz', 1017)]
 [('ypwvxc', 710) ('mcytqw', 423) ('mcaurl', 871)]
 [('sxpbdx', 911) ('hnogfk', 906) ('qopmnv', 181)]
 [('sqeise', 704) ('mcytqw', 423) ('mcaurl', 871)]
 [('mymknv', 978) ('fwuzyl', 838) ('qopmnv', 181)]
 [('zsgtmt', 712) ('agnmek', 545) ('mqcgtl', 747)]
 [('gnfvip', 774) ('uebqjr', 455) ('lyhxdu', 771)]]
[2005, 2004, 1998, 1998, 1997, 2004, 2000]
74
